# Convolutional Neural Networks

So far we have been treating images as flattened arrays of data. One might argue that such representation is not the best, since by flattening 2D images we are losing all spatial information. Let's now use a different network that exploits spatial information by using convolutional layers.

In [1]:
import matplotlib.pyplot as plt  
%matplotlib inline
from utils import plot_samples, plot_curves
import time

Using TensorFlow backend.


In [2]:
import numpy as np
# force random seed for results to be reproducible
SEED = 4242
np.random.seed(SEED)

## MNIST

In the first part of this session we will use MNIST data to train the network. Using fully connected layers we achieved an accuracy of nearly 0.98 on the test set. Let's see if we can improve this with convolutional layers.

In [3]:
from keras.datasets import mnist
from keras.utils import np_utils

# the data, shuffled and split between train and test sets
(X_train, y_train), (X_test, y_test) = mnist.load_data()

img_rows = X_train.shape[1]
img_cols = X_train.shape[2]

X_train = X_train.reshape(X_train.shape[0], img_rows, img_cols, 1)
X_test = X_test.reshape(X_test.shape[0], img_rows, img_cols, 1)
input_shape = (img_rows, img_cols, 1)

X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train /= 255
X_test /= 255
print('X_train shape:', X_train.shape)
print(X_train.shape[0], 'train samples')
print(X_test.shape[0], 'test samples')


nb_classes = 10
# convert class vectors to binary class matrices
Y_train = np_utils.to_categorical(y_train, nb_classes)
Y_test = np_utils.to_categorical(y_test, nb_classes)

('X_train shape:', (60000, 28, 28, 1))
(60000, 'train samples')
(10000, 'test samples')


**Exercise:** Design a network with with 2 [convolutional layers](https://keras.io/layers/convolutional/#convolution2d), a fully connected layer and a classifier to train on MNIST. Remember to use non liniarities and max pooling layers. Check keras documentation to learn how to use these layers.

In [5]:
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.layers import Convolution2D, MaxPooling2D, Flatten


# number of convolutional filters to use
nb_filters = 32
# size of pooling area for max pooling
pool_size = (2, 2)
# convolution kernel size
kernel_size = (3,3)

dense_hdim = 128 # dimensionality of the first fully connected layer
dr_ratio = 0.2
model = Sequential()
# TODO: add layers here
model.add(Convolution2D(nb_filters,kernel_size[0],kernel_size[1],
                        border_mode='same',input_shape=input_shape))
model.add(Activation('relu'))
model.add(Convolution2D(nb_filters,kernel_size[0],kernel_size[1],
                        border_mode='same'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=pool_size))
model.add(Flatten())
model.add(Dropout(dr_ratio))
model.add(Dense(dense_hdim))
model.add(Activation('relu'))
model.add(Dropout(dr_ratio))
model.add(Dense(10))
model.add(Activation('softmax'))

model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
convolution2d_3 (Convolution2D)  (None, 28, 28, 32)    320         convolution2d_input_2[0][0]      
____________________________________________________________________________________________________
activation_3 (Activation)        (None, 28, 28, 32)    0           convolution2d_3[0][0]            
____________________________________________________________________________________________________
convolution2d_4 (Convolution2D)  (None, 28, 28, 32)    9248        activation_3[0][0]               
____________________________________________________________________________________________________
activation_4 (Activation)        (None, 28, 28, 32)    0           convolution2d_4[0][0]            
___________________________________________________________________________________________

**Exercise**: Calculate the number of parameters of the new network. 

**Exercise**: The convolutional layer in keras has a parameter ```border_mode```, which can take values of 'valid' (no padding) or 'same' (+padding). What is the impact in the parameters when setting it to  'valid' or 'same', respectively? Why does the number of parameters change?

Let's train the model:

In [ ]:
from keras.optimizers import SGD
lr = 0.01
# For now we will not decrease the learning rate
decay = 0

optim = SGD(lr=lr, decay=decay, momentum=0.9, nesterov=True)

In [ ]:
batch_size = 128
nb_epoch = 30
verbose = 2

model.compile(loss='categorical_crossentropy',
              optimizer=optim,
              metrics=['accuracy'])
t = time.time()
history = model.fit(X_train, Y_train,
                batch_size=batch_size, nb_epoch=nb_epoch,
                verbose=verbose,validation_data=(X_test, Y_test))
print (time.time() - t, "seconds.")

score = model.evaluate(X_test, Y_test, verbose=0)
print ("-"*10)
print ("Loss: %f"%(score[0]))
print ("Accuracy: %f"%(score[1]))
plot_curves(history,nb_epoch)

In [ ]:
model.save('../models/mnist_conv.h5')

## Part 2: CIFAR

At this point we already know how to train a convnet for classification. Let's now switch to a more challenging dataset of colour images: CIFAR 10.

In [ ]:
from keras.datasets import cifar10
import numpy as np
np.random.seed(4242)

Let's load the dataset and display some samples:

In [ ]:
# The data, shuffled and split between train and test sets:
(X_train, y_train), (X_test, y_test) = cifar10.load_data()
print('X_train shape:', X_train.shape)
print(X_train.shape[0], 'train samples')
print(X_test.shape[0], 'test samples')

plot_samples(X_train,5)

We format the data before training:

In [ ]:
nb_classes = 10
# Convert class vectors to binary class matrices.
Y_train = np_utils.to_categorical(y_train, nb_classes)
Y_test = np_utils.to_categorical(y_test, nb_classes)

X_train.shape
img_rows = X_train.shape[1]
img_cols = X_train.shape[2]
input_shape = (img_rows, img_cols,3)

X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train /= 255
X_test /= 255

input_shape

**Exercise:** Design and train a convnet to classify cifar 10 images. It should be VERY similar to the one you just designed for MNIST...

In [ ]:
n_filters = 32
f_size = 3
p_size = 2
dp_ratio = 0.5
dense_dim = 128

model = Sequential()
# add layers 
model.summary()

**Exercise**: Calculate the number of parameters of the model. 

Now let's train the model. We will train for 100 epochs this time.We will use this trained model in the next lab session to visualize its weights and activations. Do not worry if training is not finished at that time; we have a pretrained model ready for you.

In [ ]:
batch_size=256
nb_epoch=100
lr = 0.01
decay = 1e-6

optim = SGD(lr=lr, decay=decay, momentum=0.9, nesterov=True)

model.compile(loss='categorical_crossentropy',
              optimizer=optim,
              metrics=['accuracy'])
t = time.time()

history = model.fit(X_train, Y_train,
                batch_size=batch_size, nb_epoch=nb_epoch,
                verbose=0,validation_data=(X_test, Y_test))

print (time.time() - t, 'seconds.')

In [ ]:
score = model.evaluate(X_test, Y_test, verbose=0)
print ("Loss: %f"%(score[0]))
print ("Accuracy: %f"%(score[1]))
plot_curves(history,nb_epoch)

In [ ]:
model.save('../models/cifar10.h5')